In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from sklearn.metrics import classification_report, confusion_matrix
from timm import create_model
from tqdm import tqdm

# Paths
model_path = "adaptive_efficientnetv2_rw_s_emotion_model.pth"  # Path to the best saved model
data_dir = "../DATA_PREPARE_ATT_04/Grayscale_Face_images"  # Path to the dataset

# Configuration
batch_size = 32
num_classes = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Emotion categories
emotion_classes = ["Anger", "Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Data Preparation for Testing
def prepare_test_loader(data_dir, batch_size):
    transform_test = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((260, 260)),  # Size used during training
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
    ])
    
    print("Loading test dataset...")
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    print("Test dataset loaded.")
    return test_loader, test_dataset

# Load Model
def load_model(model_path, num_classes):
    print("Loading the saved model...")
    model = create_model('efficientnetv2_rw_s', pretrained=True, num_classes=num_classes, in_chans=1)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    print("Model loaded successfully.")
    return model

# Evaluate Model
def evaluate_model(model, test_loader, classes):
    print("\nEvaluating on Test Data...")
    model.eval()
    test_loss = 0.0
    correct, total = 0, 0
    all_preds, all_labels = [], []

    # Loss function (optional, can be removed if not required for testing)
    criterion = nn.CrossEntropyLoss()

    with tqdm(total=len(test_loader), desc="Testing Batches", unit="batch") as pbar:
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                # Collect predictions and labels for metrics
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                pbar.update(1)

    test_loss /= len(test_loader)
    test_accuracy = correct / total
    print(f"\nTest Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=classes))

    # Confusion Matrix
    print("\nConfusion Matrix:")
    conf_matrix = confusion_matrix(all_labels, all_preds)
    print(conf_matrix)

    return test_accuracy, test_loss, conf_matrix

# Main Script
if __name__ == "__main__":
    test_loader, test_dataset = prepare_test_loader(data_dir, batch_size)
    model = load_model(model_path, num_classes)
    evaluate_model(model, test_loader, emotion_classes)


c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Loading test dataset...
Test dataset loaded.
Loading the saved model...


C:\Users\Tuf\AppData\Local\Temp\ipykernel_11148\1202509668.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=dev

Model loaded successfully.

Evaluating on Test Data...


Testing Batches: 100%|██████████| 604/604 [02:02<00:00,  4.94batch/s]


Test Loss: 0.8178, Test Accuracy: 71.65%

Classification Report:
              precision    recall  f1-score   support

       Anger       0.63      0.54      0.59      2361
    Contempt       0.83      0.80      0.81      2552
     Disgust       0.72      0.79      0.75      2423
        Fear       0.58      0.63      0.60      2281
       Happy       0.88      0.90      0.89      2508
     Neutral       0.74      0.79      0.76      2507
         Sad       0.58      0.57      0.58      2253
    Surprise       0.74      0.68      0.71      2417

    accuracy                           0.72     19302
   macro avg       0.71      0.71      0.71     19302
weighted avg       0.72      0.72      0.72     19302


Confusion Matrix:
[[1285   96  289  206   44  117  252   72]
 [ 100 2035  173   33    5   50   91   65]
 [ 126   74 1903  102   11   23  125   59]
 [ 138   42   84 1437   41   93  224  222]
 [  18   13    2   11 2259  143   24   38]
 [  94   34   26   61  128 1980  143   41]
 [ 220